In [ ]:
import subprocess
import shlex
import os

from subprocess import PIPE
from pathlib import Path

In [ ]:
CC_PATTERN = re.compile(r"(?<!^)(?=[A-Z])")

In [ ]:
def submit_independent_spatially_explicit(
    delta_t, step_slice, dedup_cache, event_slice, # sweetspot
    repeats=10, walltime=1, speciation=0.000001, sample=0.00025, memory=16,
    reporters=['Execution()', 'Biodiversity()'], output='./STDIN'
):
    walltime = "{:02d}".format(walltime)
    reporters = ','.join(reporters)
    
    config = fr"""
    #PBS -lselect={1}:ncpus={1}:mem={memory}gb
    #PBS -J 0-{max(repeats - 1, 0)}
    #PBS -lwalltime={walltime}:00:00

    $HOME/necsim-rust/target-base/release/rustcoalescence simulate '(
        speciation: {speciation},
        sample: {sample},
        seed: '$(python3 -c "import random; print(random.getrandbits(64))")',

        algorithm: Independent(
            delta_t: {delta_t},
            step_slice: {step_slice},
            dedup_cache: {dedup_cache},
            parallelism_mode: Monolithic(event_slice: {event_slice}),
        ),

        scenario: SpatiallyExplicit(
            habitat: "'$HOME'/necsim-rust/maps/madingley/fg0size12/habitat.tif",
            dispersal: "'$HOME'/necsim-rust/maps/madingley/fg0size12/dispersal.tif",
        ),

        reporters: [
            Plugin(
                library: "'$HOME'/necsim-rust/target-base/release/deps/libnecsim_plugins_common.so",
                reporters: [{reporters}],
            ),
        ],
    )'
    """
    
    output = Path(output).resolve(strict=False)
    output.parent.mkdir(parents=True, exist_ok=True)
    
    cwd = os.getcwd()
    os.chdir(output.parent)
    
    # Submit the simulation batch
    result = subprocess.run(shlex.split(
        f"qsub -N {output.name}"
    ), check=True, input=config, stdout=PIPE, universal_newlines=True).stdout.strip()
    
    os.chdir(cwd)
    
    return result

In [ ]:
def submit_cuda_spatially_explicit(
    delta_t, block_size, grid_size, step_slice, dedup_cache, event_slice, # sweetspot
    repeats=10, walltime=1, speciation=0.000001, sample=0.00025,
    reporters=['Execution()', 'Biodiversity()'], output='./STDIN'
):
    walltime = "{:02d}".format(walltime)
    reporters = ','.join(reporters)
    
    config = fr"""
    #PBS -lselect={1}:ncpus={4*1}:mem={24*1}gb:ngpus={1}:gpu_type=P100
    #PBS -J 0-{max(repeats - 1, 0)}
    #PBS -lwalltime={walltime}:00:00

    $HOME/necsim-rust/target-base/release/rustcoalescence simulate '(
        speciation: {speciation},
        sample: {sample},
        seed: '$(python3 -c "import random; print(random.getrandbits(64))")',

        algorithm: CUDA(
            device: {0},
            ptx_jit: {str(True).lower()},
            delta_t: {delta_t},
            block_size: {block_size},
            grid_size: {grid_size},
            step_slice: {step_slice},
            dedup_cache: {dedup_cache},
            parallelism_mode: Monolithic(event_slice: {event_slice}),
        ),

        scenario: SpatiallyExplicit(
            habitat: "'$HOME'/necsim-rust/maps/madingley/fg0size12/habitat.tif",
            dispersal: "'$HOME'/necsim-rust/maps/madingley/fg0size12/dispersal.tif",
        ),

        reporters: [
            Plugin(
                library: "'$HOME'/necsim-rust/target-base/release/deps/libnecsim_plugins_common.so",
                reporters: [{reporters}],
            ),
        ],
    )'
    """
    
    output = Path(output).resolve(strict=False)
    output.parent.mkdir(parents=True, exist_ok=True)
    
    cwd = os.getcwd()
    os.chdir(output.parent)
    
    # Submit the simulation batch
    result = subprocess.run(shlex.split(
        f"qsub -N {output.name}"
    ), check=True, input=config, stdout=PIPE, universal_newlines=True).stdout.strip()
    
    os.chdir(cwd)
    
    return result

In [ ]:
def submit_monolithic_spatially_explicit(
    algorithm,
    repeats=10, walltime=1, speciation=0.000001, sample=0.00025, memory=16,
    reporters=['Execution()', 'Biodiversity()'], output='./STDIN'
):
    walltime = "{:02d}".format(walltime)
    reporters = ','.join(reporters)
    
    config = fr"""
    #PBS -lselect={1}:ncpus={1}:mem={memory}gb
    #PBS -J 0-{max(repeats - 1, 0)}
    #PBS -lwalltime={walltime}:00:00

    $HOME/necsim-rust/target-base/release/rustcoalescence simulate '(
        speciation: {speciation},
        sample: {sample},
        seed: '$(python3 -c "import random; print(random.getrandbits(64))")',

        algorithm: {algorithm}(
            parallelism_mode: Monolithic,
        ),

        scenario: SpatiallyExplicit(
            habitat: "'$HOME'/necsim-rust/maps/madingley/fg0size12/habitat.tif",
            dispersal: "'$HOME'/necsim-rust/maps/madingley/fg0size12/dispersal.tif",
        ),

        reporters: [
            Plugin(
                library: "'$HOME'/necsim-rust/target-base/release/deps/libnecsim_plugins_common.so",
                reporters: [{reporters}],
            ),
        ],
    )'
    """
    
    output = Path(output).resolve(strict=False)
    output.parent.mkdir(parents=True, exist_ok=True)
    
    cwd = os.getcwd()
    os.chdir(output.parent)
    
    # Submit the simulation batch
    result = subprocess.run(shlex.split(
        f"qsub -N {output.name}"
    ), check=True, input=config, stdout=PIPE, universal_newlines=True).stdout.strip()
    
    os.chdir(cwd)
    
    return result

In [ ]:
for speciation in [0.01, 0.001, 0.0001, 0.00001, 0.000001, 0.0000001, 0.00000001]:
    for algorithm in ["Classical", "Gillespie", "SkippingGillespie"]:
        submit_monolithic_spatially_explicit(
            algorithm,
            walltime=6, repeats=10, speciation=speciation, sample=0.00025,
            output=f"{CC_PATTERN.sub('-', algorithm).lower()}/pbs.{speciation}",
        )
    
    submit_independent_spatially_explicit(
        2.0, 10.0, f"Relative(factor: {1.0})", """event_slice,""" # delta_t, step_slice, dedup_cache,
        walltime=6, repeats=10, speciation=speciation, sample=0.00025,
        output=f"independent/pbs.{speciation}",
    )
    
    submit_cuda_spatially_explicit(
        """delta_t, block_size, grid_size, step_slice, dedup_capacity, event_slice,""" # sweetspot
        walltime=6, repeats=10, speciation=speciation, sample=0.00025,
        output=f"cuda/pbs.{speciation}",
    )